# **Capstone Project Predictive Modelling:**
### **By James Kavanagh**

### Objective:
The objective of this analysis is to use the most effective features of the data set as determined in objective 1 to build an effective predictive model for future weekly sales for use in refining inventory processess and there by boosting company revenue as a whole. A comparison will be performed between 3 different types of models including regression, categoric, and time series to determine which of the 3 is most effective over all.

The features that have been chosen are as follows:



### Internal influences:
   - Clearance, a type of markdown referring to clearance sales. 
   - Rollback, a type of markdown reffering to product returns.
   - Member, a type of markdown reffering to new member discounts.

### External influences:
   - Season, reffering to weather related seasonal change.
   - Holiday, reffering to holiday periods.
   - CPI, reffering to the consumer price index which is a basic measure of the publics willingness to spend money.
     

In [1]:
# The first step is of course the importation of all the different packages required for predictive modelling.

# To access the data and for basic statistical analysis we need:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For linear regression modelling:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

# For Categoric modelling:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# For Time Series modelling:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

### **The Data:**

In [2]:
# First we need to access the data:
df1 = pd.read_csv("Raw_Data_v2.csv", index_col ='index')
df1.head()

,store_num,store_name,date,season,is_holiday,dept_num,dept_name,weekly_sales,sales_category,termperature,...,clearance_v,reduction_v,buyone_getone_v,rollback_v,member_v,total_markdowns,CPI,unemployment,store_type,store_size
index,,,,,,,,,,,,,,,,,,,,,
1,1,Anytown,11/11/2011,Autumn,0,55,Media and Gaming,23728.53,High Average,15.06,...,10382.9,6115.67,215.07,2406.62,6551.42,25671.68,218.0,0.079,3,62881
2,1,Anytown,11/11/2011,Autumn,0,91,Frozen Foods,67041.24,High,15.06,...,10382.9,6115.67,215.07,2406.62,6551.42,25671.68,218.0,0.079,3,62881
3,1,Anytown,11/11/2011,Autumn,0,44,Fabrics and Crafts,5859.12,Low,15.06,...,10382.9,6115.67,215.07,2406.62,6551.42,25671.68,218.0,0.079,3,62881
4,1,Anytown,11/11/2011,Autumn,0,26,Infants/Toddlers,7693.46,Low Average,15.06,...,10382.9,6115.67,215.07,2406.62,6551.42,25671.68,218.0,0.079,3,62881
5,1,Anytown,11/11/2011,Autumn,0,14,Kitchen and Dining,14903.78,Average,15.06,...,10382.9,6115.67,215.07,2406.62,6551.42,25671.68,218.0,0.079,3,62881


In [3]:
# And we double check the data types to make sure they are correct:
df1["date"] = pd.to_datetime(df1["date"])
pd.DataFrame(df1.dtypes)

C:\Users\jbk29\AppData\Local\Temp\ipykernel_14584\3710984722.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df1["date"] = pd.to_datetime(df1["date"])


,0
store_num,int64
store_name,object
date,datetime64[ns]
season,object
is_holiday,int64
dept_num,int64
dept_name,object
weekly_sales,float64
sales_category,object
termperature,float64


In [4]:
# And we can also check the correlation matrix to confirm our feature choices:
df1.corr()
pd.DataFrame(df1.corr()['weekly_sales'].sort_values(ascending=False))

C:\Users\jbk29\AppData\Local\Temp\ipykernel_14584\3486382195.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df1.corr()
C:\Users\jbk29\AppData\Local\Temp\ipykernel_14584\3486382195.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  pd.DataFrame(df1.corr()['weekly_sales'].sort_values(ascending=False))


,weekly_sales
weekly_sales,1.000000
store_type,0.182563
dept_num,0.147292
total_markdowns,0.125650
member_v,0.090362
clearance_v,0.086428
buyone_getone_v,0.062100
rollback_v,0.061604
reduction_v,0.032227
is_holiday,0.017890


In [5]:
# In order to make the model as precise as possible we need to create dummy columns for each unique store and department combination:
combo_dummies = pd.get_dummies(df1[['store_name', 'dept_name']], prefix=['store_name', 'dept_name'])
combo_dummies.sample(n=5, random_state=1)

,store_name_Anytown,store_name_Bayview,store_name_Beachside,store_name_Bridgewater,store_name_Brookside,store_name_Brookville,store_name_Cedarville,store_name_Cityville,store_name_Crestwood,store_name_Greenfield,...,dept_name_Service Deli,dept_name_Shoes,dept_name_Sporting Goods,dept_name_Stationary,dept_name_Storage and Organisation,dept_name_Toys,dept_name_Wireless,"dept_name_Wireless, Cameras and Supplies",dept_name_Womens Outerwear,dept_name_Womens Socks
index,,,,,,,,,,,,,,,,,,,,,
121444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75398,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30754,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# In order to use season as a predictive feature we need to create some dummy columns to represent this categorical data in numerical form:
season_dummies = pd.get_dummies(df1.season, prefix='season')
season_dummies.sample(n=5, random_state=1)

,season_Autumn,season_Spring,season_Summer,season_Winter
index,,,,
121444,0,0,1,0
14533,0,0,0,1
118316,0,0,1,0
75398,0,1,0,0
30754,0,0,0,1


In [7]:
# We only realy need 3 dummy columns as the presence of the first 3 define the presence of the 4th:
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)

In [8]:
# Now we need to join our 2 new dummy tables to our original data set:
df2 = pd.concat([df1, combo_dummies, season_dummies], axis=1)
df2.sample(n=5, random_state=1)

,store_num,store_name,date,season,is_holiday,dept_num,dept_name,weekly_sales,sales_category,termperature,...,dept_name_Stationary,dept_name_Storage and Organisation,dept_name_Toys,dept_name_Wireless,"dept_name_Wireless, Cameras and Supplies",dept_name_Womens Outerwear,dept_name_Womens Socks,season_Spring,season_Summer,season_Winter
index,,,,,,,,,,,,,,,,,,,,,
121444,40,Oak Park,2012-08-17,Summer,0,46,Beauty,9499.90,Low Average,20.78,...,0,0,0,0,0,0,0,0,1,0
14533,39,Mountain Brook,2011-09-12,Winter,0,40,Pharmacy,60748.59,High,9.64,...,0,0,0,0,0,0,0,0,0,1
118316,37,Riverdale,2012-10-08,Summer,0,81,Commercial Bread,9966.18,Low Average,29.12,...,0,0,0,0,0,0,0,0,1,0
75398,13,Brookside,2012-04-05,Spring,0,26,Infants/Toddlers,12389.70,Low Average,12.45,...,0,0,0,0,0,0,0,1,0,0
30754,12,Greenfield,2012-01-20,Winter,0,16,Lawn and Garden,3204.91,Low,7.89,...,0,0,0,0,0,0,0,0,0,1


In [9]:
column_names = df2.columns.tolist()
print(column_names)

['store_num', 'store_name', 'date', 'season', 'is_holiday', 'dept_num', 'dept_name', 'weekly_sales', 'sales_category', 'termperature', 'fuel_price', 'clearance_v', 'reduction_v', 'buyone_getone_v', 'rollback_v', 'member_v', 'total_markdowns', 'CPI', 'unemployment', 'store_type', 'store_size', 'store_name_Anytown', 'store_name_Bayview', 'store_name_Beachside', 'store_name_Bridgewater', 'store_name_Brookside', 'store_name_Brookville', 'store_name_Cedarville', 'store_name_Cityville', 'store_name_Crestwood', 'store_name_Greenfield', 'store_name_Greenwood', 'store_name_Harmony', 'store_name_Hillcrest', 'store_name_Hillside', 'store_name_Hinsdale', 'store_name_Lakeshore', 'store_name_Lakeside', 'store_name_Mountain Brook', 'store_name_Mountaintop', 'store_name_Mountainview', 'store_name_Oak Park', 'store_name_Oakwood', 'store_name_Oceanview', 'store_name_Paradise Valley', 'store_name_Pinecrest', 'store_name_Pleasantville', 'store_name_Ridgewood', 'store_name_Riverbend', 'store_name_Riverdale

### **The Regression Model:**

In [9]:
# First we need to define our list of features and our X and y variables for the model:
feature_cols = ['clearance_v', 'rollback_v', 'member_v', 'season_Spring', 'season_Summer', 'season_Winter', 'is_holiday', 'CPI',]
X = df2[feature_cols]
y = df2.weekly_sales

# Next we fit our variables to the model:
linreg = LinearRegression()
linreg.fit(X, y)

# Then we look at the y intercept and the coefficients for each feature to see what it tells us:
print(linreg.intercept_)
list(zip(feature_cols, linreg.coef_))

15042.087484705688


[('clearance_v', 0.279147929364823),
 ('rollback_v', -0.12353776650128619),
 ('member_v', 0.3166441511044382),
 ('season_Spring', -84.80783290198822),
 ('season_Summer', -121.11681463146245),
 ('season_Winter', -113.71491262503666),
 ('is_holiday', 2188.9976630234264),
 ('CPI', -11.78961102859176)]

In [33]:
# This indicates that holding all other features fixed for this model:
# Every $1 increase in the value of clearance markdowns is associated with a 28c increase in weekly sales
# Every $1 increase in the value of rollback  markdowns is associated with a 12c decrease in weekly sales
# Every $1 increase in the value of member markdowns is associated with a 32c increase in weekly sales
# The spring season is associated with an average weekly decrease of $85 in weekly sales 
# The summer season is associated with an average weekly decrease of $121 in weekly sales 
# The winter season is associated with an average weekly decrease of $114 in weekly sales 
# Holiday periods are associated with an average weekly increase of $2189 in weekly sales
# Every 1 unit increase in CPI is associated with a $12 decrease in weekly sales 

In [10]:
# Finally we can test the accuracy of our model by splitting our data into 2 parts, a training set and a test:
def train_test_rmse(df, feature_cols):
    
    X = df2[feature_cols]
    y = df2.weekly_sales
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123)
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    y_pred = linreg.predict(X_test)
    print (np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# We can then compare the 2 to find our models root means square error RMSE for whichever features we include:
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "member_v", "season_Spring", "season_Summer", "season_Winter", "is_holiday", "CPI"]))
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "member_v", "is_holiday", "CPI"]))
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "member_v", "CPI"]))
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "member_v", "is_holiday"]))
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "member_v"]))
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "CPI"]))
print (train_test_rmse(df2, ["clearance_v","is_holiday"]))
print (train_test_rmse(df2, ["is_holiday", "CPI"]))
print (train_test_rmse(df2, ["season_Spring", "season_Summer", "season_Winter", "is_holiday", "CPI"]))
print (train_test_rmse(df2, ["clearance_v", "rollback_v", "member_v", "season_Spring", "season_Summer", "season_Winter", "CPI"]))
print (train_test_rmse(df2, ["season_Spring", "season_Summer", "season_Winter"]))
print (train_test_rmse(df2, ["clearance_v", "member_v", "is_holiday"]))
print (train_test_rmse(df2, ['clearance_v', 'rollback_v', 'member_v', 'season_Spring', 'season_Summer', 'season_Winter', 'is_holiday', 'CPI',
                'store_name_Anytown', 'store_name_Bayview', 'store_name_Beachside', 'store_name_Bridgewater', 'store_name_Brookside', 
                'store_name_Brookville', 'store_name_Cedarville', 'store_name_Cityville', 'store_name_Crestwood', 'store_name_Greenfield', 
                'store_name_Greenwood', 'store_name_Harmony', 'store_name_Hillcrest', 'store_name_Hillside', 'store_name_Hinsdale', 
                'store_name_Lakeshore', 'store_name_Lakeside', 'store_name_Mountain Brook', 'store_name_Mountaintop', 
                'store_name_Mountainview', 'store_name_Oak Park', 'store_name_Oakwood', 'store_name_Oceanview', 'store_name_Paradise Valley', 
                'store_name_Pinecrest', 'store_name_Pleasantville', 'store_name_Ridgewood', 'store_name_Riverbend', 'store_name_Riverdale', 
                'store_name_Riverside', 'store_name_Rivertown', 'store_name_Rockville', 'store_name_Sandy Springs', 'store_name_Seaview', 
                'store_name_Southlake', 'store_name_Springdale', 'store_name_Springfield', 'store_name_Sunnydale', 'store_name_Sunrise', 
                'store_name_Sunset', 'store_name_Valleyview', 'store_name_Westlake', 'store_name_Willow Creek', 'store_name_Willowbrook', 
                'store_name_Woodland', 'dept_name_1 Hour Photo', 'dept_name_Auto Service', 'dept_name_Automotive', 
                'dept_name_Bath and Shower', 'dept_name_Beauty', 'dept_name_Bedding', 'dept_name_Books and Magazines', 'dept_name_Boys Wear', 
                'dept_name_Candy and Tobacco', 'dept_name_Celebrations and Cards', 'dept_name_Commercial Bread', 'dept_name_Concept Stores', 
                'dept_name_Customer Service', 'dept_name_Dairy', 'dept_name_Dry Grocery', 'dept_name_Electronics', 
                'dept_name_Fabrics and Crafts', 'dept_name_Foundations', 'dept_name_Fresh Bakery', 'dept_name_Fresh Meat', 
                'dept_name_Fresh Produce', 'dept_name_Frozen Foods', 'dept_name_Furniture and Luggage', 'dept_name_Gasoline', 
                'dept_name_Girls Wear', 'dept_name_Grocery', 'dept_name_Handbags and accessories', 'dept_name_Hardware', 'dept_name_Hearing', 
                'dept_name_Home Décor', 'dept_name_Hosiery', 'dept_name_Household Chemicles', 'dept_name_Impulse Merchandise', 
                'dept_name_Infant Consumables and Hardlines', 'dept_name_Infants/Toddlers', 'dept_name_Intimate Apparel', 
                'dept_name_Jewellery', 'dept_name_Kitchen and Dining', 'dept_name_Ladies Wear', 'dept_name_Large Appliances', 
                'dept_name_Lawn and Garden', 'dept_name_Liquor', 'dept_name_Live Paints', 'dept_name_Media and Gaming', 'dept_name_Mens Wear', 
                'dept_name_Office and Store Supplies', 'dept_name_Optical', 'dept_name_Paint and Accessories', 'dept_name_Paper Goods', 
                'dept_name_Personal Care', 'dept_name_Pets and Supplies', 'dept_name_Pharmacy', 'dept_name_Plus Size and Maternity', 
                'dept_name_Prepackaged Deli', 'dept_name_Seafood', 'dept_name_Seasonal', 'dept_name_Service Deli', 'dept_name_Shoes', 
                'dept_name_Sporting Goods', 'dept_name_Stationary', 'dept_name_Storage and Organisation', 'dept_name_Toys', 
                'dept_name_Wireless', 'dept_name_Wireless, Cameras and Supplies', 'dept_name_Womens Outerwear', 'dept_name_Womens Socks']))

22888.462809883054
None
22888.433058679773
None
22893.649067292652
None
22895.324551508318
None
22900.510565123896
None
22968.495550312142
None
22972.835022385403
None
23065.58602839424
None
23063.841180396063
None
22892.145755197496
None
23070.939640164896
None
22898.968948187572
None
13932.644954032012
None


In [12]:
# Can also compare this to the rmse of a null model to further test its effectiveness:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
y_null = np.zeros_like(y_test, dtype=float)
y_null.fill(y_test.mean())
np.sqrt(metrics.mean_squared_error(y_test, y_null))

23073.872427929775

In [65]:
# After testing the model with a variety of different feature combinations the seems less then helpful overall. 
# Simply leaving out the seasonal features resulted in the smallest level of error in the mode with a $22,888.43 descrepency.
# After retesting the model with the store/dept combination columns included the model preformed better with the previous error cut by half.
# It now returned a descrepency of $13,932.65 but Copmared to the Null models rmse of a $1.54 descrepency. . . 
# Even with the most effective combination of features this model is not a very accurate sales predictor. 

### **The Categoric Model:**

In [9]:
# For this model I'll be using the K nearest naighbours or KNN method: 
# The KNN method requires that we map the categories for the target we are trying to predict to a set of numbers:
df2['sales_category_num'] = df2.sales_category.map({'High' : 0, 'High Average': 1, 'Average': 2, 'Low Average': 3, 'Low': 4})

In [10]:
# For clarity we'll reiterate our most effective features and required variables:
feature_cols = ['clearance_v', 'rollback_v', 'member_v', 'is_holiday', 'CPI', 'season_Spring', 'season_Summer', 'season_Winter',
                'store_name_Anytown',  'store_name_Anytown', 'store_name_Bayview', 'store_name_Beachside', 'store_name_Bridgewater', 
                'store_name_Brookside', 'store_name_Brookville', 'store_name_Cedarville', 'store_name_Cityville', 
                'store_name_Crestwood', 'store_name_Greenfield', 'store_name_Greenwood', 'store_name_Harmony', 
                'store_name_Hillcrest', 'store_name_Hillside', 'store_name_Hinsdale', 'store_name_Lakeshore', 'store_name_Lakeside', 
                'store_name_Mountain Brook', 'store_name_Mountaintop', 'store_name_Mountainview', 'store_name_Oak Park', 
                'store_name_Oakwood', 'store_name_Oceanview', 'store_name_Paradise Valley', 'store_name_Pinecrest', 
                'store_name_Pleasantville', 'store_name_Ridgewood', 'store_name_Riverbend', 'store_name_Riverdale', 
                'store_name_Riverside', 'store_name_Rivertown', 'store_name_Rockville', 'store_name_Sandy Springs', 
                'store_name_Seaview', 'store_name_Southlake', 'store_name_Springdale', 'store_name_Springfield', 
                'store_name_Sunnydale', 'store_name_Sunrise', 'store_name_Sunset', 'store_name_Valleyview', 'store_name_Westlake', 
                'store_name_Willow Creek', 'store_name_Willowbrook', 'store_name_Woodland', 'dept_name_1 Hour Photo', 
                'dept_name_Auto Service', 'dept_name_Automotive', 'dept_name_Bath and Shower', 'dept_name_Beauty', 
                'dept_name_Bedding', 'dept_name_Books and Magazines', 'dept_name_Boys Wear', 'dept_name_Candy and Tobacco', 
                'dept_name_Celebrations and Cards', 'dept_name_Commercial Bread', 'dept_name_Concept Stores', 
                'dept_name_Customer Service', 'dept_name_Dairy', 'dept_name_Dry Grocery', 'dept_name_Electronics', 
                'dept_name_Fabrics and Crafts', 'dept_name_Foundations', 'dept_name_Fresh Bakery', 'dept_name_Fresh Meat', 
                'dept_name_Fresh Produce', 'dept_name_Frozen Foods', 'dept_name_Furniture and Luggage', 'dept_name_Gasoline', 
                'dept_name_Girls Wear', 'dept_name_Grocery', 'dept_name_Handbags and accessories', 'dept_name_Hardware', 
                'dept_name_Hearing', 'dept_name_Home Décor', 'dept_name_Hosiery', 'dept_name_Household Chemicles', 
                'dept_name_Impulse Merchandise', 'dept_name_Infant Consumables and Hardlines', 'dept_name_Infants/Toddlers', 
                'dept_name_Intimate Apparel', 'dept_name_Jewellery', 'dept_name_Kitchen and Dining', 'dept_name_Ladies Wear', 
                'dept_name_Large Appliances', 'dept_name_Lawn and Garden', 'dept_name_Liquor', 'dept_name_Live Paints', 
                'dept_name_Media and Gaming', 'dept_name_Mens Wear', 'dept_name_Office and Store Supplies', 'dept_name_Optical', 
                'dept_name_Paint and Accessories', 'dept_name_Paper Goods', 'dept_name_Personal Care', 'dept_name_Pets and Supplies', 'dept_name_Pharmacy', 'dept_name_Plus Size and Maternity', 
                'dept_name_Prepackaged Deli', 'dept_name_Seafood', 'dept_name_Seasonal', 'dept_name_Service Deli', 'dept_name_Shoes', 
                'dept_name_Sporting Goods', 'dept_name_Stationary', 'dept_name_Storage and Organisation', 'dept_name_Toys', 
                'dept_name_Wireless', 'dept_name_Wireless, Cameras and Supplies', 'dept_name_Womens Outerwear', 
                'dept_name_Womens Socks', 'dept_name_Service Deli']
X = df2[feature_cols]
y = df2.sales_category_num

In [11]:
# Next we once again split the data into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

In [12]:
# Then we fit the training data to create our model:
knnc = KNeighborsClassifier(n_neighbors = 1)
knnc.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [13]:
# Now we can test the model using the test set:
y_pred = knnc.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.3473506260235617

In [14]:
# So far the model is accurate around 85% of the time which isn't bad at all: 
# We can try to find a better k value on which to base the model in order to make it as accurate as possible.
score =[]

for k in range(1, 101):
    knnc = KNeighborsClassifier(n_neighbors = k)
    knnc.fit(X_train, y_train)
    y_pred = knnc.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    score.append([k, accuracy])

score

[[1, 0.3473506260235617],
 [2, 0.2822917216968672],
 [3, 0.35376934861852183],
 [4, 0.37970838396111783],
 [5, 0.38607427756352686],
 [6, 0.3940250409424692],
 [7, 0.40789265148713616],
 [8, 0.4163981192878652],
 [9, 0.4202546357440964],
 [10, 0.4249564160811453],
 [11, 0.42865444555972315],
 [12, 0.43612974800570553],
 [13, 0.44133340377199004],
 [14, 0.4429182735485234],
 [15, 0.44735590892281685],
 [16, 0.45097469491256803],
 [17, 0.4545142374134925],
 [18, 0.45472555338369697],
 [19, 0.4586613133287548],
 [20, 0.4602990120978393],
 [21, 0.46235934280733265],
 [22, 0.4628876327328438],
 [23, 0.46423477204289715],
 [24, 0.46354799513973266],
 [25, 0.46412911405779494],
 [26, 0.46724602461831055],
 [27, 0.46790638702519943],
 [28, 0.46843467695071056],
 [29, 0.46801204501030164],
 [30, 0.4680384595065772],
 [31, 0.46938559881663056],
 [32, 0.4686195784246394],
 [33, 0.4692535263352528],
 [34, 0.46893655237994614],
 [35, 0.4702044482011728],
 [36, 0.4708912251043373],
 [37, 0.470785567

In [15]:
score_table = pd.DataFrame(score).rename(columns = {0:'k', 1:'accuracy'}).sort_values('accuracy', ascending = False)
score_table.head()

,k,accuracy
56,57,0.482434
57,58,0.482038
64,65,0.481695
62,63,0.481457
59,60,0.481404


In [18]:
# As with the regression model we can also test against a null model:
most_frequent_class = y.value_counts().idxmax()
null_accuracy = y.value_counts()[most_frequent_class] / len(y)
dummy_classifier = DummyClassifier(strategy="most_frequent")
dummy_classifier.fit(X, y)
dummy_predictions = dummy_classifier.predict(X)
null_accuracy_score = accuracy_score(y, dummy_predictions)
print (null_accuracy_score)

0.48241454910454856


In [19]:
# We can see that if we set the models k value to 56 its predictive accuracy inreases from 34.74% to a maximum of 48.24%.
# Compare this to the null model's accuracy of  48.24
# This is a descent improvement but shows that at best the model can only match a prediction made with no features considered.
# Either way a 50/50 chance of producing an accurate prediction means this model is insufficent.

### **The Time Series Model:**

In [82]:
# In order to factor in previously observed seasonal change I'll be using the Sarima time series model:
# First we need to filter the data to a specific store and department to account for irregularities in our date range.
df3 = df2[(df2["store_name"] == "Anytown") & (df2["dept_num"] == 55)]

In [83]:
df3 = df3.sort_values("date")
df3.head()

,store_num,store_name,date,season,is_holiday,dept_num,dept_name,weekly_sales,sales_category,termperature,...,dept_name_Stationary,dept_name_Storage and Organisation,dept_name_Toys,dept_name_Wireless,"dept_name_Wireless, Cameras and Supplies",dept_name_Womens Outerwear,dept_name_Womens Socks,season_Spring,season_Summer,season_Winter
index,,,,,,,,,,,,,,,,,,,,,
8950,1,Anytown,2011-02-12,Winter,0,55,Media and Gaming,15958.05,Average,9.39,...,0,0,0,0,0,0,0,0,0,1
11943,1,Anytown,2011-09-12,Winter,0,55,Media and Gaming,17699.32,Average,6.63,...,0,0,0,0,0,0,0,0,0,1
1,1,Anytown,2011-11-11,Autumn,0,55,Media and Gaming,23728.53,High Average,15.06,...,0,0,0,0,0,0,0,0,0,0
2970,1,Anytown,2011-11-18,Autumn,0,55,Media and Gaming,16338.81,Average,16.81,...,0,0,0,0,0,0,0,0,0,0
5929,1,Anytown,2011-11-25,Autumn,1,55,Media and Gaming,30868.94,High,15.63,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# We also need to ensure that the date column is set as the index for our data:
df3.set_index("date", inplace=True)

In [85]:
df3.tail(12)

,store_num,store_name,season,is_holiday,dept_num,dept_name,weekly_sales,sales_category,termperature,fuel_price,...,dept_name_Stationary,dept_name_Storage and Organisation,dept_name_Toys,dept_name_Wireless,"dept_name_Wireless, Cameras and Supplies",dept_name_Womens Outerwear,dept_name_Womens Socks,season_Spring,season_Summer,season_Winter
date,,,,,,,,,,,,,,,,,,,,,
2012-08-24,1,Anytown,Summer,0,55,Media and Gaming,7482.47,Low Average,25.37,3.62,...,0,0,0,0,0,0,0,0,1,0
2012-08-31,1,Anytown,Summer,0,55,Media and Gaming,10045.55,Low Average,26.94,3.64,...,0,0,0,0,0,0,0,0,1,0
2012-09-03,1,Anytown,Spring,0,55,Media and Gaming,13415.73,Low Average,14.87,3.67,...,0,0,0,0,0,0,0,1,0,0
2012-09-14,1,Anytown,Autumn,0,55,Media and Gaming,7408.83,Low Average,23.87,3.72,...,0,0,0,0,0,0,0,0,0,0
2012-09-21,1,Anytown,Autumn,0,55,Media and Gaming,10502.56,Low Average,21.04,3.72,...,0,0,0,0,0,0,0,0,0,0
2012-09-28,1,Anytown,Autumn,0,55,Media and Gaming,9595.03,Low Average,24.49,3.67,...,0,0,0,0,0,0,0,0,0,0
2012-10-02,1,Anytown,Winter,1,55,Media and Gaming,17331.87,Average,8.90,3.41,...,0,0,0,0,0,0,0,0,0,1
2012-10-08,1,Anytown,Summer,0,55,Media and Gaming,7861.01,Low Average,29.47,3.49,...,0,0,0,0,0,0,0,0,1,0
2012-10-19,1,Anytown,Autumn,0,55,Media and Gaming,9438.28,Low Average,19.98,3.59,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# One last time we of course need to split the data into training and test sets:
train = df3.weekly_sales["2011-11-11":"2012-10-26"]
test = df3.weekly_sales["2012-08-24":"2012-10-26"]

In [87]:
# We make one last adjustment to our date range:
train.index = train.index.to_period('W-FRI').to_timestamp('W-FRI')
test.index = test.index.to_period('W-FRI').to_timestamp('W-FRI')

In [89]:
# We once again define our variables, this time for the chosen attributes of our date range:
p = 7
d = 0
q = 8
P = 1
D = 0
Q = 0

# And we fit and train our model:
seasonal_model = SARIMAX(train, order = (p, d, q), seasonal_order = (P, D, Q, 52)).fit()
y_pred_seasonal = seasonal_model.predict(start = "2012-08-24", end = "2012-10-26")
                                         
# Finally, like the regression model we check the models accuracy using its RMSE score:
sm_rmse = np.sqrt(mean_squared_error(test, y_pred_seasonal))
sm_rmse

C:\Users\jbk29\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jbk29\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jbk29\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\jbk29\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\

2990.5553376437692

In [90]:
# Finally we can compare the result to a null model rsme:
null_predictions = test.shift(1)
null_predictions = null_predictions[1:]
test = test[1:]
null_rmse = np.sqrt(mean_squared_error(test, null_predictions))
null_rmse

4910.649161366889

In [25]:
# While our time series model is still not that good it is at least better then the regression model.
# Unfortunatly it wasn't able to exceed the null modal either. 
# Time series is definitly the best model of the 3 and with more time and testing could still be further improved.
# To that end, I doscovered quite a bit of variation in the models accuracy when changing the store and department combination.
# I even discovered that in some cases it actually did exceed null modal. 
# With 45 stores and 99 departments there is a lot more testing that can be done with the many different combinations this offers.
# I attempted to use a for-loop to iterate the model over all of these combinations however,
# The nature of the date column in the data means that. . . 
# The date ranges for the train and test sets need to be adjusted manually every time we change the store and department combination.
# I have been able to incorporate this into the loop so for now, as I said, I beleave more testing can and should still be done.